In [1]:
# Print library version
import qiskit
import qiskit_aer
import qiskit_ibm_runtime

print(f"Qiskit Version: {qiskit.__version__}")
print(f"Qiskit-aer Version: {qiskit_aer.__version__}")
print(f"Qiskit-ibm-runtime Version: {qiskit_ibm_runtime.__version__}")


Qiskit Version: 1.3.2
Qiskit-aer Version: 0.16.0
Qiskit-ibm-runtime Version: 0.34.0


In [2]:
#Import all the needed library

import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.primitives import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService, IBMBackend
from math import gcd
from random import randint
from fractions import Fraction


In [3]:
# Load the ibm token if needed

def get_backend_service() -> QiskitRuntimeService:
    file = open("token_loaded.txt", "r")
    token_loaded = bool(file.read())
    file.close()
    if not token_loaded:
        file = open("token.txt", "r")
        ibm_token = file.read()
        QiskitRuntimeService.save_account( ibm_token, channel = 'ibm_quantum', overwrite=True )
        file = open("token_loaded.txt", "w")
        file.write("True")
        file.close()
    return QiskitRuntimeService()


In [4]:
# load ibm service
ibm_service = get_backend_service()


In [5]:
# Print available backend

print("\nBackend disponibili:")
for available_backend in ibm_service.backends():
    print(f"- {available_backend.name}: {available_backend.status().pending_jobs} jobs in coda")



Backend disponibili:
- ibm_brisbane: 62 jobs in coda
- ibm_kyiv: 11 jobs in coda
- ibm_sherbrooke: 86 jobs in coda


In [6]:
# select the least busy quantum computer

def get_least_busy_backend(service: QiskitRuntimeService) -> IBMBackend:
    """Returns the backend with the smallest number of pending jobs."""
    backends = service.backends()  # Get all available backends
    least_busy_backend = backends[0]
    min_pending_jobs = least_busy_backend.status().pending_jobs

    for backend in backends[1:]:
        status = backend.status()
        pending_jobs = status.pending_jobs

        if pending_jobs < min_pending_jobs:
            min_pending_jobs = pending_jobs
            least_busy_backend = backend

    return least_busy_backend


In [7]:
# print current least busy backend

current_least_busy_backend = get_least_busy_backend(ibm_service)
print(f"Backend: {current_least_busy_backend.name}, Pending Jobs: {current_least_busy_backend.status().pending_jobs}")


Backend: ibm_kyiv, Pending Jobs: 16


In [8]:
# shor number theory

def period_to_factor(N:int, a:int , r:int ):
    if r % 2 == 0 and pow(a, r // 2, N) not in [-1 % N, 1]:
        factor1 = gcd(pow(a, r // 2, N) + 1, N)
        factor2 = gcd(pow(a, r // 2, N) - 1, N)
        return factor1, factor2
    else:
        raise Exception("Not a valid period")


In [9]:
# find a (coprime number for N)

def find_a(N:int) -> int:
    while True:
        a = randint(2, N - 1)  # Random integer in [2, N-1]
        d = gcd(a, N)
        if d == 1:
            return a  # Valid a found


In [10]:
# uses the calculated quantum phase to calculate the period

def phase_to_period(phase:float, N:int) -> int:
    fraction = Fraction(phase).limit_denominator(N)
    return fraction.denominator
